# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment,Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

import azureml.core
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig

import os
import joblib
import pandas as pd
import numpy as np

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Initialize Workspace and Create an Azure ML experiment

In [ ]:
ws = Workspace.from_config()
experiment_name = 'heart-failure-automl-experiment'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

## Dataset

In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
dataset_key = "Heart-Failure-Prediction-Dataset"
description_text = "Heart Failure Prediction Dataset"

if dataset_key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[dataset_key]
else:
    # Create AML Dataset and register it into Workspace
    dataset_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(dataset_url)        
    #Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                                name=key,
                                description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

## Create or Attach an AmlCompute cluster

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "cpu-cluster"

# Check if the compute target exists
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=5)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())

## Training Configuration

In [ ]:
train, test = train_test_split(df, random_state=0, test_size=0.2)
print(train.head(3))
print(train.shape)
train.to_csv('train.csv', index=False)

In [ ]:
datastore = ws.get_default_datastore()
print(datastore)
datastore.upload_files(files = ['./train.csv'])
train = Dataset.Tabular.from_delimited_files(path = [(datastore,'train.csv')])

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
automl_settings = {
    "experiment_timeout_minutes" : 30,
    "iteration_timeout_minutes": 5,
    "max_cores_per_iteration": -1,
    "max_concurrent_iterations": 4,
    "enable_early_stopping" : True,
    "primary_metric": 'accuracy',
    "n_cross_validations": 3,
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(
    task="classification",
    compute_target=compute_target,
    training_data= train,
    label_column_name="DEATH_EVENT",
    debug_log = 'automl_errors.log',
    **automl_settings)

In [ ]:
#Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
remote_run

In [ ]:
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_automl_best_model = remote_run.get_output()
best_automlrun_metrics=automl_run.get_metrics()
print('Best Run Id:', best_run.id, sep='\n')
print('Best Run Accuracy:', best_automlrun_metrics['accuracy'], sep='\n')
print('Best Run Metrics:', best_run.get_metrics(), sep='\n')
print('Best Run Properties:', best_run.get_properties(), sep='\n')
print('Best Model:', best_run.properties['model_name'], sep='\n')
print('Fitted Automl Best Model:', fitted_automl_best_model, sep='\n')

In [ ]:
for primary_metric in best_automlrun_metrics:
    metric=best_automlrun_metrics[primary_metric]
    print(primary_metric,metric)

In [ ]:
#Save the best model
os.makedirs('./outputs', exist_ok=True)

joblib.dump(fitted_automl_best_model, filename='outputs/automl.joblib')
print(best_run)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

In [ ]:
description = 'Heart Failure Prediction AutoML Model'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)
print(model.name, model.id, model.version, sep='\t')

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script='inference/score.py')

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'type': "automl_classification"}, 
                                               description = 'Heart Data Service')

deploy_service_name = 'automl-model-deployment'
service = Model.deploy(ws, deploy_service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(show_output = True)

# Enable Application Insights
service.update(enable_app_insights=True)

print('Service State:', service.state, sep='\n')
print('Service Scoring URI:', service.scoring_uri, sep='\n')
print('Service Swagger URI:', service.swagger_uri, sep='\n')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

scoring_uri = aci_service.scoring_uri

data = {"data":
        [
          { 
            "age": 80,
            "anaemia": 1,
            "creatinine_phosphokinase": 146,
            "diabetes": 1,
            "ejection_fraction": 20,
            "high_blood_pressure": 0,
            "platelets": 127000,
            "serum_creatinine": 9.4,
            "serum_sodium": 137,
            "sex": 1,
            "smoking": 1,
            "time": 104
          },
          {
            "age": 42,
            "anaemia": 1,
            "creatinine_phosphokinase": 111,
            "diabetes": 0,
            "ejection_fraction": 38,
            "high_blood_pressure": 1,
            "platelets": 87000,
            "serum_creatinine": 0.8,
            "serum_sodium": 116,
            "sex":0,
            "smoking": 0,
            "time": 13
          },
      ]
    }

input_data = json.dumps(data)
headers = {'Content-Type': 'application/json'}
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print("\nService Logs:\n",service.get_logs())

## Cleanup

In [ ]:
service.delete()
model.delete()
compute_target.delete()